In [1]:
import numpy as np
import pandas as pd
import sqlite3
import networkx as nx

Load in data from a sql file

In [3]:
cx = sqlite3.connect('Clubhouse_Dataset_v1.db')
df = pd.read_sql_query("SELECT user_id, invited_by_user_profile FROM user", cx)
cx.commit()
cx.close()
df.head()

,user_id,invited_by_user_profile
0,4,null
1,5,null
2,8,null
3,10,null
4,12,null


I can construct an adjacency matrix out of this to create a network that's purely a tree

In [4]:
G = nx.DiGraph()

In [19]:
df.shape

(1300515, 2)

In [21]:
G.add_nodes_from(list(df.user_id[df.invited_by_user_profile == "null"]))

In [48]:
invited_users = df[df.invited_by_user_profile != "null"]
invited_users = invited_users[["invited_by_user_profile", "user_id"]]
invited_users.head()

,invited_by_user_profile,user_id
758,4,924
885,3712,1122
907,4,1150
910,658,1159
1063,843,1372


In [49]:
G.add_edges_from(list(invited_users.to_records(index=False)))

The inviter points to the invitee. 

In [51]:
A = nx.adjacency_matrix(G)

In [54]:
A[0]

<1x1387036 sparse matrix of type '<class 'numpy.int64'>'
	with 94 stored elements in Compressed Sparse Row format>

In [57]:
import scipy.sparse as sps
from sklearn.decomposition import NMF
from time import perf_counter as pc

In [58]:
model = NMF(n_components=50, init='random', random_state=0, verbose=True)

start_time = pc()
W = model.fit_transform(A)
end_time = pc()

violation: 1.0
violation: 2.4515839503942343
violation: 0.6789664339535688
violation: 0.16675330078659364
violation: 0.07087539835365612
violation: 0.049065207915235734
violation: 0.028022450258062406
violation: 0.012539289173823505
violation: 0.007957582994436389
violation: 0.005733953871539183
violation: 0.004422446260048363
violation: 0.003594947263595929
violation: 0.0030474389225726927
violation: 0.0026764516350247084
violation: 0.0024124886682194754
violation: 0.00220308618250206
violation: 0.0020133626450922833
violation: 0.001826713423506082
violation: 0.001639112265955028
violation: 0.0014508180379691836
violation: 0.0012658959609367817
violation: 0.0010924118664484177
violation: 0.0009373646000404563
violation: 0.0008039892858840406
violation: 0.0006922333610288159
violation: 0.0006000988154002191
violation: 0.000524797292060911
violation: 0.0004634653677213709
violation: 0.00041351352209605184
violation: 0.0003736251367642584
violation: 0.00034103815164826957
violation: 0.00

In [63]:
H = model.components_

W is the dictionary which is 1387036 x 50


H is the coordinates which is 50 x 1387036



uhhhhhh ok now what...